# ARN
## 算法原理简介
论文地址：[\[2001.03905\] Few-shot Action Recognition with Permutation-invariant Attention (ECCV 2020)](https://link.springer.com/chapter/10.1007/978-3-030-58558-7_31)

1. ARN是用于少样本动作分类的网络，实现少样本分类的关键就在于相似度学习模块的构建。这一模块采用relation network，自动学习度量函数，输入查询集（query set）样本的二维特征图和支持集（support set）中每一类样本的二维特征图沿着channel维度作级联后得到的特征对，即可输出每一个样本对的相似度得分。具体的，这一相似度度量模块主要由四层Conv3d构成，进行线性变换。

2. ARN包含特征编码器、自监督学习模块、注意力机制、基于增强和对齐的注意力机制、特征图聚合、少样本学习六个主要部分，如下：

- 特征编码器

特征提取对任何一个深度神经网络模型都是至关重要的一步，特征提取的准确性和针对性直接影响了模型对输入数据的采样和感知能力，一定程度上决定了模型表现的“上限”。ARN采用基于C3D结构的特征编码器Conv-4-64，即4层卷积、64输出通道，是对C3D的简化和优化。输入视频数据 **V** （如20帧）经过编码器特征提取操作 **f** 后输出维度为 **C\*T\*H\*W** 的特征向量 **Φ** 。

具体而言，编码器由四层卷积块组成，前两层卷积块计算后分别插入一层kernel和stride均为2\*2\*2的最大池化计算使输入数据在 **T H W** 维度上均缩小为原来的1/4。其中，每个卷积块依此包含卷积核大小为3\*3\*3且输出通道为64的卷积计算、批标准化（Batch Normalization）处理和ReLU激活函数处理。

- 自监督学习模块

将自监督学习模块应用于视频少样本分类，主要是为了鼓励基本网络模型学习附加的辅助任务，从而促使模型学到通用、稳定的特征提取方法，以更好地理解输入图像或视频等数据中蕴含的语义等信息。ARN的工作同时在时域和空域上引入自监督学习任务，在基于数据增强的设计思路指引下创建辅助任务（pretext）的学习。

具体的，在时域上，不同于以往工作对每一帧进行乱序操作，是对不重叠的、固定长度的帧块进行乱序操作，即随机打乱动作块在时域上的先后顺序。这样做的好处在于更不容易拆解、打破判别性强的动作块所在相关联的帧，更好地保留了连贯的动作行为信息。在空域上，可以对视频中每一帧图像进行拼图或旋转的增强操作，如拼图操作将每一帧图像分为不重叠的四个区块再随机打乱，旋转操作则是将每一帧图像随机旋转0°、90°、180°或270°。

- 注意力机制

ARN中通过注意力机制来来实现为不同贡献度的动作块分配不同关注度，即不同的注意力权重向量。具体的，考虑到计算资源的限制，时空注意力单元被分为时间和空间两个模块，分别对每一帧图像的不同部分和不同时间段的动作块的不同贡献度作出描述。

将维度为 **C\*T\*H\*W** 的特征向量 输入到时间注意力单元，得到 **1\*T\*1\*1** 维度的注意力向量，输入到空间注意力单元则得到 **1\*1\*H\*W** 维度的注意力向量。时间和空间注意力作用影响程度通过两个参数来进行调控。其中 **t** 、 **s** 分别表示时间、空间注意力单元， **Φ\*** 表示注意力特征图。

- 基于增强和对齐的注意力机制

基于增强和对齐的注意力机制（Augmentation-guided attention by alignment）是ARN工作中的主要创新点，模型实现的序列无关效果也主要得益于这个机制。如下图中所示，这一机制主要通过基于数据增强的自监督任务来加强时间和空间的注意力。具体的，通过将1）原始数据经过特征编码得到的表征先经过注意力单元的处理、再进行增强操作（如拼图或旋转）得到的“增强了的注意力向量”，2）增强后的视频数据经过特征编码得到的表征经由注意力单元处理后进一步得到注意力向量（即“增强数据的注意力向量”），这两个向量进行对齐学习来实施。这样打乱和对齐的做法旨在通过数据增强建立动作块和特定注意力权重之间的映射，使注意力机制能够适应输入序列的变化，调节、拓宽模型感受的范围，从而达到序列无关的提升效果。

- 特征图聚合

ARN中的聚合阶段采用带有功率归一化（Power Normalization, PN）的二阶池化（Second-order Pooling）操作。二阶池化通过算子 **g** 对特征向量进行降维操作得到特征图 **ψ** 。

具体的，经过特殊编码器提取特征和注意力单元增强特征后得到特征向量维度为 **C\*T\*H\*W** ，在将这些向量输入到二阶池化函数之前首先对其进行变形，成为 **C\*N** ，其中 **N = T\*H\*W** ，便于后续池化计算。经过二阶池化的处理（核心为向量乘上本身的转置向量，即一个整形操作）输出得到维度为 **C\*C** 的特征向量，即二维特征图，在实现降维的同时也通过 **N** 即 **T\*H\*W** 维度上的信息，体现了对输入数据的序列无关性，且对于输入视频帧长也不具约束。

特别的，此处二阶池化操作还带有Power Normalization。从直观上看，加上PN操作的好处在于可以降低频繁出现的视觉元素对结果的贡献，而增加出现频率较低的视觉元素对结果的贡献，这也更加符合人类观察学习的特点。PN关注图像中的视觉元素是否共同出现，而非统计共同出现了几次，这大大降低了后续比较器需要记忆的数据量，而这也正是学习分类任务的自然特性之一。所以，PN引入二阶池化很适合少样本任务的学习。

- 少样本学习————关系网络

ARN中的比较器采用关系网络（Relation Network, RN）的结构，这一部分也是该网络结构适用于少样本学习的直接体现。在ARN网络结构中，比较器RN的输入为关系描述（relation descriptors）向量，即一对对的特征对（Feature Pair）。这些特征对由二阶池化后得到的二维特征图经过在通道维度上的进行级联拼接得到，维度为 **2\*C\*C** 。具体的，将拼接操作记为 **θ** ，是指查询集中的特征图和支持集中每一类别的特征图分别做通道维度上的级联的过程。特别的，如果不是n-way 1shot任务，即支持集中每一个类别包含不止一个样本，则先对这些样本在通道维度上做平均（或最大）级联，得到与各个样本具有相同维度 **C\*C** 的特征图代表这一类别的图像特征，再与查询集中样本做上述级联。

这些拼接好的特征对输入到RN（记为 **r** ）中进行相似度的比较，输出相似度得分高的类别作为预测类别从而实现对视频的分类。

RN采用均方误差（Mean Square Error, MSE）作为损失函数对网络进行训练。


3. ARN网络模型的总体框架图如下：

![ARN_architecture](./pics/ARN_model.png)

4. ARN实现视频少样本动作分类的可视化效果如下：

![1](./pics/result-1.gif)
![2](./pics/result-2.gif)
![3](./pics/result-3.gif)
![4](./pics/result-4.gif)
![5](./pics/result-5.gif)

### 环境准备
```text
git clone https://gitee.com/yanlq46462828/zjut_mindvideo.git
cd zjut_mindvideo

# Please first install mindspore according to instructions on the official website: https://www.mindspore.cn/install

pip install -r requirements.txt
pip install -e .
```
### 训练流程


In [ ]:
from mindspore import nn
from mindspore import context, load_checkpoint, load_param_into_net, ParallelMode
from mindspore.communication.management import init, get_rank, get_group_size
from mindspore.nn import MSELoss
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor

from msvideo.utils.check_param import Validator,Rel

##### 数据集准备

下载[UCF101](https://www.crcv.ucf.edu/data/UCF101.php)数据集及其split文件至如"/data0/ucf101-videos"的路径下并解压。

- UCF101数据集来自于Youtube，包含13320个动作视频段和101个动作类别。在这项工作中，数据集中的70个类被用于训练，10个类用于验证，剩余21个类用于测试。

```text
.
└─ucf101                                    // contains 101 file folder
  ├── ApplyEyeMakeup                        // contains 145 videos
  │   ├── v_ApplyEyeMakeup_g01_c01.avi      // video file
  │   ├── v_ApplyEyeMakeup_g01_c02.avi      // video file
  │    ...
  ├── ApplyLipstick                         // contains 114 image files
  │   ├── v_ApplyLipstick_g01_c01.avi       // video file
  │   ├── v_ApplyLipstick_g01_c02.avi       // video file
  │    ...
  ├── ucfTrainTestlist                      // contains category files
  │   ├── classInd.txt                      // Category file.
  │   ├── testlist01.txt                    // split file
  │   ├── trainlist01.txt                   // split file
  ...
```

##### 数据集加载

通过基于VideoDataset编写的Ucf101类来加载ucf101数据集。

In [ ]:
from msvideo.data.ucf101 import UCF101
# Data Pipeline.
dataset_train = UCF101(path="/data0/ucf101-videos",
                               batch_size=1,
                               split='train',
                               shuffle=False,
                               seq=16,
                               num_parallel_workers=2,
                               suffix="task",
                               task_num=100,
                               task_n=5,
                               #    task_k=5,
                               task_k=1,
                               task_q=1)
ckpt_save_dir = './arn_output'

##### 数据处理

通过VideoReshape、VideoResize、VideoNormalize等一系列变换对数据进行预处理。

In [ ]:
from msvideo.data.transforms.video_to_tensor import VideoToTensor
from msvideo.data.transforms import VideoResize, VideoReshape, VideoNormalize

transforms = [
    VideoReshape((-1, 240, 320, 3)),
    VideoResize((128, 128)),
    VideoToTensor(),
    VideoNormalize((0.3474, 0.3474, 0.3474), (0.2100, 0.2100, 0.2100)),
    VideoReshape((3, -1, 16, 128, 128))
    ]

dataset_train.transform = transforms
dataset_train = dataset_train.run()


Validator.check_int(dataset_train.get_dataset_size(), 0, Rel.GT)
step_size = dataset_train.get_dataset_size()

##### 网络构建

ARN包含embedding、backbone、neck、head几部分，大致可以归纳为如下几点：

- embedding部分：主要是一个类似于C3D结构的特征提取器，通过四层Conv3d进行线性变换。这一部分支持对C3D主干网络的复用，也支持直接使用Unit3d类（包含Conv3d、norm、activation）进行构建，仅需在入参is_c3d中指定True或False即可。

- backbone部分：主要使用空间注意力模块对上一步embedding提取得到的特征进行增强，后将特征维度变换为C * N便于后续处理。空间注意力模块主要由三层Conv3d构成，进行线性变换。

- neck部分：主要进行带有功率归一化（power normalization）的二阶池化（second-order pooling）处理，有助于实现序列无关以及少样本条件下的性能可靠度。后将特征维度变换为C * C便于特征图的拼接和特征对的构建，以实现最后基于度量的分类。

- head部分：主要相似度学习模块，采用relation network实现，自动学习度量函数。具体如上第一点中所述。

In [10]:
from msvideo.models.arn import ARN
# Create model
network = ARN()
# network = ARN(support_num_per_class = 1,
#               query_num_per_class = 1,
#               class_num = 5)

In [11]:
from msvideo.schedule.lr_schedule import warmup_cosine_annealing_lr_v1
# Set learning rate scheduler.
learning_rate = warmup_cosine_annealing_lr_v1(lr=0.01,
                                                steps_per_epoch=step_size,
                                                warmup_epochs=4,
                                                max_epoch=100,
                                                t_max=100,
                                                eta_min=0)

In [12]:
# Define loss function.
network_loss = MSELoss()

# Define optimizer.
network_opt = nn.Adam(network.trainable_params(),
                      learning_rate=learning_rate
                      )

In [13]:
# set checkpoint config for the network
ckpt_config = CheckpointConfig(save_checkpoint_steps=step_size,
                               keep_checkpoint_max=10
                               )
ckpt_callback = ModelCheckpoint(prefix="arn",
                                directory=ckpt_save_dir, 
                                config=ckpt_config
                                )

In [14]:
from msvideo.utils.task_acc import TaskAccuracy

# Init the model.
model = Model(network, loss_fn=network_loss,
              optimizer=network_opt, 
              metrics={"Accuracy": TaskAccuracy()}
              )

In [ ]:
from msvideo.utils.callbacks import SaveCallback

# Begin to train.
print("[Start training arn.]")
print("=" * 80)
model.train(1,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor(),
                       SaveCallback(model)],
            dataset_sink_mode=False)
print("[End of training arn.]")

训练结果（以100 step为例）如下：

### 评估流程

In [ ]:
from mindspore import context, load_checkpoint, load_param_into_net
from mindspore.nn import MSELoss
from mindspore.train import Model

from msvideo.models.arn import ARN
from msvideo.data.ucf101 import UCF101
from msvideo.data.transforms.video_to_tensor import VideoToTensor
from msvideo.data.transforms import VideoResize, VideoReshape, VideoNormalize

from msvideo.utils.task_acc import TaskAccuracy

In [ ]:
context.set_context(mode=context.PYNATIVE_MODE,
                        # mode=context.GRAPH_MODE,
                        device_target="GPU", device_id=3)

# Data Pipeline.
dataset_eval = UCF101(path="/data0/ucf101-videos",
                          batch_size=1,
                          split='test',
                          shuffle=False,
                          seq=16,
                          num_parallel_workers=1,
                          suffix="task",
                          task_num=100,
                          task_n=5,
                          # task_k=5,
                          task_k=1,
                          task_q=1
                          )

In [25]:
transforms = [
    VideoReshape((-1, 240, 320, 3)),
    VideoResize((128, 128)),
    VideoToTensor(),
    VideoNormalize((0.3474, 0.3474, 0.3474), (0.2100, 0.2100, 0.2100)),
    VideoReshape((3, -1, 16, 128, 128))
]
dataset_eval.transform = transforms
dataset_eval = dataset_eval.run()

下载模型权重文件[arn.ckpt](https://zjuteducn-my.sharepoint.com/:u:/g/personal/201906010313_zjut_edu_cn/ER55hujI22BOkyjL5UrBVt0BfKx8lmeW5DRctx46tfZRkA?e=hdIZIu)。

In [26]:
# Create model.
network = ARN()
# network = ARN(support_num_per_class=5)

param_dict = load_checkpoint('/home/teddy20/arn/arn.ckpt')
load_param_into_net(network, param_dict)

# Define loss function.
network_loss = MSELoss()

# Init the model.
model = Model(network, loss_fn=network_loss,
              metrics={"Accuracy": TaskAccuracy()})


In [ ]:
# Begin to eval.
print('[Start eval arn]')
result = model.eval(dataset_eval, dataset_sink_mode=False)
print("result:", result["Accuracy"])

评估结果如下：

## Code
代码仓库地址如下：

Gitee   https://gitee.com/yanlq46462828/zjut_mindvideo

Github  https://github.com/ZJUT-ERCISS/arn_mindspore